In [1]:
import psycopg2 as pg
import psycopg2.extras as pgExtras

In [2]:
params = {  'user': 'cristiano', 'password': 'cristiano',
            'host':'localhost', 'port':'5432', 'database':'afterqualifying'}

def reopenConn(conn, cur, params):
    conn.commit()
    cur.close()
    conn.close()
    conn = pg.connect(**params)
    cur = conn.cursor()

    return conn, cur

class Station:
    def __init__(self, idStation, edgeId, positionInEdge):
        self.idStation = idStation
        self.edgeId = edgeId
        self.positionInEdge = positionInEdge

def loadStations():
    setStations = set()
    stations = list()
    fileName = 'finalResult.sol'

    with open(fileName) as file:
        for line in file:
            if not line.startswith('pos') and not line.startswith('#'):
                lineList = line.split()
                valueVar = round(float(lineList[1]))
                if valueVar == 1:
                    setStations.add(lineList[0])

    with open(fileName) as file:
        countStations = 0
        for line in file:
            lineList = line.split()
            keyWithoutPos = lineList[0][4:]

            if line.startswith('pos') and keyWithoutPos in setStations:
                countStations += 1
                edgeId = lineList[0].split('-')[2]
                positionInEdge = float(lineList[1])
                station = Station(idStation=countStations, edgeId=edgeId, positionInEdge=positionInEdge)
                stations.append(station)

    return stations
loadStations()

def storeStations():
    stations = loadStations()
    
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute('delete from STATION')
        
        lengthToInsert = 100000
        stationsToInsert = list()
        sqlInsert = 'insert into STATION (IDSTATION, IDEDGE_FK, POSITIONINEDGE) values %s'
        for station in stations:
            stationsToInsert.append((station.idStation, station.edgeId, station.positionInEdge))
            
            if len(stationsToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, stationsToInsert)
                stationsToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(stationsToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, stationsToInsert)
        
        cur.execute(''' update	STATION
                        set		POSITIONINEDGE =	case
                                                        when	EDGE.LENGTH <= 1 then 0
                                                        else	((POSITIONINEDGE * 100)/EDGE.LENGTH)/100
                                                    end
                        from	STREETSEGMENT EDGE
                        where	EDGE.IDEDGE = STATION.IDEDGE_FK''')

        conn.commit()
        cur.close()
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

storeStations()